# Fraud Detection — Fine-tune ResNet-50 on Product Images (Colab)

This notebook:
1. Uploads `image_dataset.csv` (contains `product_id`, `fraud_label`, `image_url`)
2. Downloads product images from URLs
3. Fine-tunes a pretrained **ResNet-50** for binary fraud classification
4. Handles class imbalance with weighted loss
5. Reports evaluation metrics on a stratified test split

**Make sure to select GPU runtime:** Runtime → Change runtime type → T4 GPU

In [ ]:
# Install dependencies (Colab-compatible — use Colab's pre-installed torch/pandas/numpy)
!pip -q install torchvision scikit-learn Pillow requests tqdm

In [ ]:
import os
import io
import warnings
import requests
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    f1_score,
    accuracy_score,
    average_precision_score,
)

warnings.filterwarnings('ignore')

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# Config
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
SEED = 42
NUM_WORKERS = 2
IMAGE_DIR = Path('./downloaded_images')
MODEL_SAVE_PATH = Path('./resnet_fraud_model')

IMAGE_DIR.mkdir(exist_ok=True)
MODEL_SAVE_PATH.mkdir(exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# Upload image_dataset.csv
if IN_COLAB:
    print('Upload image_dataset.csv ...')
    uploaded = files.upload()
    if 'image_dataset.csv' not in uploaded:
        raise ValueError(f'Expected image_dataset.csv. Uploaded: {list(uploaded.keys())}')
    df = pd.read_csv(io.BytesIO(uploaded['image_dataset.csv']))
else:
    df = pd.read_csv('processed_data/image_dataset.csv')

print(f'Dataset shape: {df.shape}')
print(f'Fraud distribution:\n{df["fraud_label"].value_counts()}\n')

In [ ]:
# Download images from URLs (parallel, with retry)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

def download_image(row):
    """Download a single image. Returns (product_id, success)."""
    pid = str(row['product_id']).zfill(6)
    url = row['image_url']
    save_path = IMAGE_DIR / f'{pid}.jpg'
    if save_path.exists():
        return pid, True
    try:
        resp = requests.get(url, headers=HEADERS, timeout=15)
        resp.raise_for_status()
        img = Image.open(io.BytesIO(resp.content)).convert('RGB')
        img.save(save_path, 'JPEG')
        return pid, True
    except Exception:
        return pid, False

print(f'Downloading {len(df)} images (skipping already downloaded) ...')
results = []
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
    for future in tqdm(as_completed(futures), total=len(futures)):
        results.append(future.result())

success_ids = {pid for pid, ok in results if ok}
failed_ids  = {pid for pid, ok in results if not ok}
print(f'\nDownloaded: {len(success_ids)} | Failed: {len(failed_ids)}')

# Keep only rows with successfully downloaded images
df['pid_str'] = df['product_id'].apply(lambda x: str(x).zfill(6))
df = df[df['pid_str'].isin(success_ids)].reset_index(drop=True)
print(f'Usable samples: {len(df)}')
print(f'Fraud distribution after filtering:\n{df["fraud_label"].value_counts()}')

In [ ]:
# Train / Test split (stratified)
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df['fraud_label']
)
train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

print(f'Train: {len(train_df)} | Test: {len(test_df)}')
print(f'Train fraud ratio: {train_df["fraud_label"].mean():.4f}')
print(f'Test  fraud ratio: {test_df["fraud_label"].mean():.4f}')

In [ ]:
# Transforms & Dataset
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

class FraudImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pid = str(row['product_id']).zfill(6)
        img_path = self.img_dir / f'{pid}.jpg'
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(row['fraud_label'], dtype=torch.long)
        return image, label

train_dataset = FraudImageDataset(train_df, IMAGE_DIR, train_transform)
test_dataset  = FraudImageDataset(test_df,  IMAGE_DIR, test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f'Train batches: {len(train_loader)} | Test batches: {len(test_loader)}')

In [ ]:
# Model: ResNet-50 (pretrained) with custom head
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Freeze early layers, fine-tune layer4 + fc
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True

# Replace classifier
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 2),
)

model = model.to(device)

# Class weights for imbalance
n_neg = (train_df['fraud_label'] == 0).sum()
n_pos = (train_df['fraud_label'] == 1).sum()
class_weights = torch.tensor([1.0, n_neg / n_pos], dtype=torch.float).to(device)
print(f'Class weights: not-fraud=1.00, fraud={n_neg / n_pos:.2f}')

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=LR, weight_decay=1e-4
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f'Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)')

In [ ]:
# Training loop
best_f1 = 0.0
history = {'train_loss': [], 'val_loss': [], 'val_f1': [], 'val_acc': []}

for epoch in range(1, EPOCHS + 1):
    # --- Train ---
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_loss = running_loss / len(train_dataset)

    # --- Evaluate ---
    model.eval()
    val_loss = 0.0
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = val_loss / len(test_dataset)
    val_acc = accuracy_score(all_labels, all_preds)
    val_f1  = f1_score(all_labels, all_preds)

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)
    history['val_acc'].append(val_acc)

    scheduler.step()

    marker = ''
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), MODEL_SAVE_PATH / 'best_resnet50.pth')
        marker = ' ← saved best'

    print(f'Epoch {epoch}/{EPOCHS}  '
          f'train_loss={train_loss:.4f}  '
          f'val_loss={val_loss:.4f}  '
          f'val_acc={val_acc:.4f}  '
          f'val_f1={val_f1:.4f}{marker}')

print(f'\nBest validation F1: {best_f1:.4f}')

In [ ]:
# Load best model & final evaluation
model.load_state_dict(torch.load(MODEL_SAVE_PATH / 'best_resnet50.pth', map_location=device))
model.eval()

all_preds, all_labels, all_probs = [], [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

y_true = np.array(all_labels)
y_pred = np.array(all_preds)
y_prob = np.array(all_probs)

acc = accuracy_score(y_true, y_pred)
f1  = f1_score(y_true, y_pred)
roc = roc_auc_score(y_true, y_prob)
ap  = average_precision_score(y_true, y_prob)

print('=' * 60)
print('  Final Evaluation — ResNet-50')
print('=' * 60)
print(f'Accuracy          : {acc:.4f}')
print(f'F1 Score (fraud)  : {f1:.4f}')
print(f'ROC-AUC           : {roc:.4f}')
print(f'Avg Precision (PR): {ap:.4f}\n')
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=['Not Fraud', 'Fraud']))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))

In [ ]:
# Export test-set predictions for ensemble
image_preds_df = pd.DataFrame({
    'product_id': test_df['product_id'].values,
    'fraud_label': y_true,
    'image_fraud_proba': y_prob,
    'image_pred': y_pred,
})
image_preds_df.to_csv('image_test_predictions.csv', index=False)
print(f'Saved image_test_predictions.csv  ({len(image_preds_df)} rows)')
print(image_preds_df.head())

# Download for local ensemble
if IN_COLAB:
    files.download('image_test_predictions.csv')

In [ ]:
# Training curves
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'],   label='Val Loss')
axes[0].set_title('Loss'); axes[0].legend()

axes[1].plot(history['val_acc'])
axes[1].set_title('Validation Accuracy')

axes[2].plot(history['val_f1'])
axes[2].set_title('Validation F1 (Fraud)')

for ax in axes:
    ax.set_xlabel('Epoch')
plt.tight_layout()
plt.show()

In [ ]:
# Save model
print(f'Best model saved to: {(MODEL_SAVE_PATH / "best_resnet50.pth").resolve()}')

# Optional: download the model file in Colab
if IN_COLAB:
    files.download(str(MODEL_SAVE_PATH / 'best_resnet50.pth'))